In [1]:
import pandas as pd

df_tweets = pd.read_csv("./SA_data/twitter_dataset.csv")
df_tweets.head()

KeyboardInterrupt: 

In [ ]:
len(df_tweets)

3142403

In [ ]:
#df_tweets["sentiment"] = df_tweets["sentiment"].replace({0.0 : "negative", 1.0 : "positive", 2.0 : "neutral"})
df_tweets.head()

,Unnamed: 0,tweet,sentiment
0,0,is upset that he can't update his Facebook by ...,0.0
1,1,@Kenichan I dived many times for the ball. Man...,0.0
2,2,my whole body feels itchy and like its on fire,0.0
3,3,"@nationwideclass no, it's not behaving at all....",0.0
4,4,@Kwesidei not the whole crew,0.0


In [ ]:
df_tweets.sentiment.value_counts()
# 0 = negative, 1 = positive, 2 = neutral

sentiment
0.0    1570067
1.0    1561529
2.0      10725
Name: count, dtype: int64

In [ ]:
# only taking a part of the data to save resources (with same label-splits as the whole dataset)

In [ ]:
df_tweets = df_tweets.dropna()

In [ ]:
# following BERT-Pytorch SA tutorial from https://www.kaggle.com/code/chayan8/sentiment-analysis-using-bert-pytorch/notebook

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df_tweets.index.values, 
                                                  df_tweets.sentiment.values, 
                                                  test_size=0.15, 
                                                  random_state=42,
                                                  stratify=df_tweets.sentiment.values)

In [ ]:
df_tweets["data_type"] = "not set"

In [ ]:
df_tweets.loc[X_train, "data_type"] = "train"
df_tweets.loc[X_val, "data_type"] = "val"

In [ ]:
df_tweets.groupby(["sentiment", "data_type"]).count()

Unnamed: 0    tweet
sentiment data_type                     
0.0       train         1334557  1334557
          val            235510   235510
1.0       train         1327299  1327299
          val            234230   234230
2.0       train            9116     9116
          val              1609     1609

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

tokenizer = BertTokenizer.from_pretrained(
    "bert-base-uncased",
    do_lower_case=True
)

c:\Users\hausb\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hausb\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\hausb\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokeniz

In [ ]:
import torch

encoded_data_train = tokenizer.batch_encode_plus(
    df_tweets[df_tweets.data_type=="train"].tweet.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors="pt"
)

encoded_data_val = tokenizer.batch_encode_plus(
    df_tweets[df_tweets.data_type=="val"].tweet.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors="pt"
)

input_ids_train = encoded_data_train["input_ids"]
attention_masks_train = encoded_data_train["attention_mask"]
labels_train = torch.tensor(df_tweets[df_tweets.data_type=="train"].sentiment.values)

input_ids_val = encoded_data_val["input_ids"]
attention_masks_val = encoded_data_val["attention_mask"]
labels_val = torch.tensor(df_tweets[df_tweets.data_type=="val"].sentiment.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\hausb\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
dataset_train = TensorDataset(input_ids_train, 
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val, 
                            attention_masks_val,
                           labels_val)

In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
                                      "bert-base-uncased", 
                                      num_labels = 3,
                                      output_attentions = False,
                                      output_hidden_states = False
                                     )

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 4

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)

In [ ]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps = len(dataloader_train)*epochs
)

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')

In [ ]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random

seed_val = 12
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train, 
                        desc='Epoch {:1d}'.format(epoch), 
                        leave=False, 
                        disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})     
    
    #torch.save(model.state_dict(), f'Models/BERT_ft_Epoch{epoch}.model')
    
    tqdm.write('\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')

In [ ]:
accuracy_per_class(predictions, true_vals)

In [ ]:
# following tutorial https://github.com/nicknochnack/BERTSentiment/blob/main/Sentiment.ipynb

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

c:\Users\hausb\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

c:\Users\hausb\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hausb\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\hausb\AppData\Local\Programs\Python\Python312\Lib\si

In [ ]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')
result = model(tokens)

In [ ]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

# BerTweet Sentiment Analysis

In [1]:
# using bertweet-sentiment analysis 
# https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis

from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")

model = AutoModelForSequenceClassification.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")

In [2]:
import torch

def sentiment_score(text, tokenizer, model):
    tokens = tokenizer.encode(text, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))

In [3]:
import pandas as pd

#df_brd_sa = pd.read_csv("./analysis_data/Brand Sentiment Analysis Dataset/Dataset - Train.csv")

# read in data after NER csv
df_brd_sa = pd.read_csv("./data after NER.csv")

df_brd_sa.head()

,Unnamed: 0,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,entities
0,0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,"[{""word"": ""."", ""entity"": ""B-person""}, {""word"":..."
1,1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,"[{""word"": ""@"", ""entity"": ""B-person""}, {""word"":..."
2,2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,"[{""word"": ""@"", ""entity"": ""B-person""}, {""word"":..."
3,3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,"[{""word"": ""\u0120festival"", ""entity"": ""B-event..."
4,4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,"[{""word"": ""xt"", ""entity"": ""I-event""}, {""word"":..."


In [4]:
df_brd_sa["is_there_an_emotion_directed_at_a_brand_or_product"].unique()

array(['Negative emotion', 'Positive emotion',
       'No emotion toward brand or product', "I can't tell"], dtype=object)

In [5]:
df_brd_sa.dropna(inplace=True)

In [133]:
df_brd_sa["sentiment_prediction"] = df_brd_sa["tweet_text"].apply(lambda x: sentiment_score(x[:500], tokenizer, model))

In [134]:
df_brd_sa["sentiment_prediction"].unique()

array([1, 2, 0], dtype=int64)

In [13]:
# 0 negative
# 1 neutral
# 2 positive
df_brd_sa["is_there_an_emotion_directed_at_a_brand_or_product"] = df_brd_sa["is_there_an_emotion_directed_at_a_brand_or_product"].replace({"Negative emotion" : 0, "Positive emotion" : 2, "No emotion toward brand or product" : 1, "I can't tell" : 1})
df_brd_sa.head()

C:\Users\hausb\AppData\Local\Temp\ipykernel_27056\2089848440.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_brd_sa["is_there_an_emotion_directed_at_a_brand_or_product"] = df_brd_sa["is_there_an_emotion_directed_at_a_brand_or_product"].replace({"Negative emotion" : 0, "Positive emotion" : 2, "No emotion toward brand or product" : 1, "I can't tell" : 1})


,Unnamed: 0,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,entities,sentiment_prediction_finetuned
0,0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,0,"[{""word"": ""."", ""entity"": ""B-person""}, {""word"":...",0
1,1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,2,"[{""word"": ""@"", ""entity"": ""B-person""}, {""word"":...",2
2,2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,2,"[{""word"": ""@"", ""entity"": ""B-person""}, {""word"":...",2
3,3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,0,"[{""word"": ""\u0120festival"", ""entity"": ""B-event...",0
4,4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,2,"[{""word"": ""xt"", ""entity"": ""I-event""}, {""word"":...",2


In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def performance_metrics(df, label, prediction):
     accuracy = accuracy_score(df[label], df[prediction])
     precision = precision_score(df[label], df[prediction], average="weighted")
     recall = recall_score(df[label], df[prediction], average="weighted")
     f1 = f1_score(df[label], df[prediction], average="weighted")

     print(f"Accuracy: {accuracy}")
     print(f"Precision: {precision}")
     print(f"Recall: {recall}")
     print(f"F1-Score: {f1}")

In [138]:
performance_metrics(df_brd_sa, "is_there_an_emotion_directed_at_a_brand_or_product", "sentiment_prediction")

Accuracy: 0.7289577635976907
Precision: 0.8810658835129812
Recall: 0.7289577635976907
F1-Score: 0.7884397587742731


In [12]:
df_brd_sa.to_csv("./data after SA.csv")

## bertweet fine-tuning

In [ ]:
# trying to fine-tune this model (goal of accuracy> and precision>)
from datasets import Dataset

dataset = Dataset.from_pandas(df_brd_sa)

In [7]:
def tokenize_function(example):
    return tokenizer(example['tweet_text'], truncation=True, padding='max_length', max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.rename_column('is_there_an_emotion_directed_at_a_brand_or_product', 'labels')
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/3291 [00:00<?, ? examples/s]

In [8]:
train_test_split = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [19]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, precision_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

In [10]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4, # adjust depending on resources
    per_device_eval_batch_size=4,
    num_train_epochs=2, # adjust depending on resources
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True
)

c:\Users\hausb\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

C:\Users\hausb\AppData\Local\Temp\ipykernel_7652\3252959399.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: hausbichler-georg (hausbichler-georg-wirtschaftsuniversit-t-wien). Use `wandb login --relogin` to force relogin


  0%|          | 0/1316 [00:00<?, ?it/s]

{'loss': 0.4696, 'grad_norm': 34.91965103149414, 'learning_rate': 1.2401215805471124e-05, 'epoch': 0.76}


  0%|          | 0/165 [00:00<?, ?it/s]

c:\Users\hausb\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 0.3217962682247162, 'eval_accuracy': 0.9044006069802731, 'eval_precision': 0.8776387724816433, 'eval_recall': 0.9044006069802731, 'eval_f1': 0.8906713195717271, 'eval_runtime': 125.3936, 'eval_samples_per_second': 5.255, 'eval_steps_per_second': 1.316, 'epoch': 1.0}
{'loss': 0.3243, 'grad_norm': 56.448238372802734, 'learning_rate': 4.80243161094225e-06, 'epoch': 1.52}


  0%|          | 0/165 [00:00<?, ?it/s]

c:\Users\hausb\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 0.3860619068145752, 'eval_accuracy': 0.91350531107739, 'eval_precision': 0.8822575684694961, 'eval_recall': 0.91350531107739, 'eval_f1': 0.8969385263946141, 'eval_runtime': 115.0542, 'eval_samples_per_second': 5.728, 'eval_steps_per_second': 1.434, 'epoch': 2.0}
{'train_runtime': 7890.3636, 'train_samples_per_second': 0.667, 'train_steps_per_second': 0.167, 'train_loss': 0.36438215539810503, 'epoch': 2.0}


TrainOutput(global_step=1316, training_loss=0.36438215539810503, metrics={'train_runtime': 7890.3636, 'train_samples_per_second': 0.667, 'train_steps_per_second': 0.167, 'total_flos': 346257257730048.0, 'train_loss': 0.36438215539810503, 'epoch': 2.0})

In [12]:
results = trainer.evaluate(test_dataset)
print(results)

  0%|          | 0/165 [00:00<?, ?it/s]

c:\Users\hausb\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 0.3217962682247162, 'eval_accuracy': 0.9044006069802731, 'eval_precision': 0.8776387724816433, 'eval_recall': 0.9044006069802731, 'eval_f1': 0.8906713195717271, 'eval_runtime': 114.8893, 'eval_samples_per_second': 5.736, 'eval_steps_per_second': 1.436, 'epoch': 2.0}


In [ ]:
model.save_pretrained('./models/bertweetSA_after_inital_finetuning')
tokenizer.save_pretrained('./models/bertweetSA_after_inital_finetuning')

('./models/bertweetSA_after_inital_finetuning\\tokenizer_config.json',
 './models/bertweetSA_after_inital_finetuning\\special_tokens_map.json',
 './models/bertweetSA_after_inital_finetuning\\vocab.txt',
 './models/bertweetSA_after_inital_finetuning\\bpe.codes',
 './models/bertweetSA_after_inital_finetuning\\added_tokens.json')

In [6]:
#loading and trying the saved model
tokenizer2 = AutoTokenizer.from_pretrained("./models/bertweetSA_after_inital_finetuning")

model2 = AutoModelForSequenceClassification.from_pretrained("./models/bertweetSA_after_inital_finetuning")

In [8]:
df_brd_sa["sentiment_prediction_finetuned"] = df_brd_sa["tweet_text"].apply(lambda x: sentiment_score(x[:500], tokenizer2, model2))

In [14]:
# very high results, but may not be too telling because it was tested on same data it was finetuned on
performance_metrics(df_brd_sa, "is_there_an_emotion_directed_at_a_brand_or_product", "sentiment_prediction_finetuned")

Accuracy: 0.9310240048617442
Precision: 0.9037352697049685
Recall: 0.9310240048617442
F1-Score: 0.9170998038436116


c:\Users\hausb\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [15]:
# using big tech companies dataset to test 
# Big Tech Companies - Tweet Sentiment https://www.kaggle.com/datasets/wjia26/big-tech-companies-tweet-sentiment/data
df_tweets_bigtech = pd.read_csv("./analysis_data/Big Tech Companies - Tweet Sentiment/Bigtech - 12-07-2020 till 19-09-2020/Bigtech - 12-07-2020 till 19-09-2020.csv")
df_tweets_bigtech.head()

,created_at,file_name,followers,friends,group_name,location,retweet_count,screenname,search_query,text,twitter_id,username,polarity,partition_0,partition_1
0,2020-07-12 09:24:26,AMD,25,114,AMD,United Kingdom,0.0,moffphcgaming,#AMD,Been on holiday so back now. Gonna try get som...,1.282244e+18,🕹MoffPHC Gaming🕹,-0.3102,Technology,AMD
1,2020-07-12 09:09:36,AMD,159,1144,AMD,digitalverse,4.0,ironparr0t,#AMD,RT @NinjaParanoid: #AMD #Ryzen #3900x #rtx2070...,1.282241e+18,ironparrot,0.0000,Technology,AMD
2,2020-07-12 08:37:31,AMD,4931,7,AMD,NaN,0.0,ASUS_ROG_IN,#AMD,The Beast #StrixGA35 comes packed with Up to A...,1.282233e+18,ASUS ROG IN,0.0000,Technology,AMD
3,2020-07-12 08:31:24,AMD,9,188,AMD,HAHA no....,0.0,XApochrypha,#AMD,Recently purchased everything for my first per...,1.282231e+18,XenosApochrypha,0.0000,Technology,AMD
4,2020-07-12 08:16:45,AMD,1719,1,AMD,digitalocean,1.0,LinuxDreams,#AMD,RT @LinuxReviews: #Linux architect Linus Torva...,1.282227e+18,LinuxDreams,-0.3612,Technology,AMD


In [8]:
len(df_tweets_bigtech)

866909

In [16]:
df_tweets_bigtech.dropna(inplace=True)

In [17]:
# removing emojis from the tweets
import regex as re
df_tweets_bigtech["text"] = df_tweets_bigtech["text"].str.replace("[^A-Za-z0-9]", "", flags=re.UNICODE)

In [61]:
# transforming polarity-column to be comparable with labels given by bertweet
min_polarity = df_tweets_bigtech["polarity"].min()
max_polarity = df_tweets_bigtech["polarity"].max()
print(min_polarity, max_polarity)

-0.9975 0.9968


In [18]:
# 0 negative
# 1 neutral
# 2 positive
# function converting polarity to sentiment
def polarity_to_sentiment(polarity):
    if -0.2 <= polarity <= 0.2:
        return 1
    if polarity > 0.2:
        return 2
    else:
        return 0

In [19]:
df_tweets_bigtech["sentiment"] = df_tweets_bigtech["polarity"].apply(polarity_to_sentiment)
df_tweets_bigtech.head()

,created_at,file_name,followers,friends,group_name,location,retweet_count,screenname,search_query,text,twitter_id,username,polarity,partition_0,partition_1,sentiment
0,2020-07-12 09:24:26,AMD,25,114,AMD,United Kingdom,0.0,moffphcgaming,#AMD,Been on holiday so back now. Gonna try get som...,1.282244e+18,🕹MoffPHC Gaming🕹,-0.3102,Technology,AMD,0
1,2020-07-12 09:09:36,AMD,159,1144,AMD,digitalverse,4.0,ironparr0t,#AMD,RT @NinjaParanoid: #AMD #Ryzen #3900x #rtx2070...,1.282241e+18,ironparrot,0.0000,Technology,AMD,1
3,2020-07-12 08:31:24,AMD,9,188,AMD,HAHA no....,0.0,XApochrypha,#AMD,Recently purchased everything for my first per...,1.282231e+18,XenosApochrypha,0.0000,Technology,AMD,1
4,2020-07-12 08:16:45,AMD,1719,1,AMD,digitalocean,1.0,LinuxDreams,#AMD,RT @LinuxReviews: #Linux architect Linus Torva...,1.282227e+18,LinuxDreams,-0.3612,Technology,AMD,0
7,2020-07-12 08:11:41,AMD,69,135,AMD,Amsterdam,1.0,LinuxReviews,#AMD,"#Linux architect Linus Torvalds: AVX512 Is ""A ...",1.282226e+18,LinuxReviews,-0.3612,Technology,AMD,0


In [90]:
df_tweets_bigtech["sentiment"].value_counts()

sentiment
1    277033
2    260709
0     76243
Name: count, dtype: int64

In [20]:
# creating a sample from the data with equally distributed sentiment labels

df_tweets_bigtech_negative = df_tweets_bigtech[df_tweets_bigtech["sentiment"]==0]
#negative_sample = df_tweets_bigtech_negative.sample(n=3500) 
# random sampling leads to error in predicting
negative_sample = df_tweets_bigtech_negative[:3500]


df_tweets_bigtech_neutral = df_tweets_bigtech[df_tweets_bigtech["sentiment"]==1]
neutral_sample = df_tweets_bigtech_neutral[:2500]

df_tweets_bigtech_positive = df_tweets_bigtech[df_tweets_bigtech["sentiment"]==2]
positive_sample = df_tweets_bigtech_positive[:3500]

df_tweets_bigtech_10ksample = pd.concat([negative_sample, neutral_sample, positive_sample], axis=0, ignore_index=True)
df_tweets_bigtech_10ksample.head()


,created_at,file_name,followers,friends,group_name,location,retweet_count,screenname,search_query,text,twitter_id,username,polarity,partition_0,partition_1,sentiment
0,2020-07-12 09:24:26,AMD,25,114,AMD,United Kingdom,0.0,moffphcgaming,#AMD,Been on holiday so back now. Gonna try get som...,1.282244e+18,🕹MoffPHC Gaming🕹,-0.3102,Technology,AMD,0
1,2020-07-12 08:16:45,AMD,1719,1,AMD,digitalocean,1.0,LinuxDreams,#AMD,RT @LinuxReviews: #Linux architect Linus Torva...,1.282227e+18,LinuxDreams,-0.3612,Technology,AMD,0
2,2020-07-12 08:11:41,AMD,69,135,AMD,Amsterdam,1.0,LinuxReviews,#AMD,"#Linux architect Linus Torvalds: AVX512 Is ""A ...",1.282226e+18,LinuxReviews,-0.3612,Technology,AMD,0
3,2020-07-12 02:22:50,AMD,34,155,AMD,San Francisco,0.0,NdrewGarcia,#AMD,#AMD stuck in a range box chart https://t.co/5...,1.282138e+18,Encino_Man,-0.2500,Technology,AMD,0
4,2020-07-11 23:58:44,AMD,802,730,AMD,"New Jersey, USA",0.0,Roger_Clinton1,#AMD,$AMD Epyc Milan Leak – Three early Genesis sam...,1.282102e+18,Roger Ocasio-Clinton,-0.3400,Technology,AMD,0


In [105]:
df_tweets_bigtech_10ksample["sentiment"].value_counts()

sentiment
0    3500
2    3500
1    2500
Name: count, dtype: int64

In [106]:
df_tweets_bigtech_10ksample["sentiment_prediction_finetuned"] = df_tweets_bigtech_10ksample["text"].apply(lambda x: sentiment_score(x[:500], tokenizer2, model2))

In [107]:
df_tweets_bigtech_10ksample.head()

,created_at,file_name,followers,friends,group_name,location,retweet_count,screenname,search_query,text,twitter_id,username,polarity,partition_0,partition_1,sentiment,sentiment_prediction_finetuned
0,2020-07-12 09:24:26,AMD,25,114,AMD,United Kingdom,0.0,moffphcgaming,#AMD,Been on holiday so back now. Gonna try get som...,1.282244e+18,🕹MoffPHC Gaming🕹,-0.3102,Technology,AMD,0,2
1,2020-07-12 08:16:45,AMD,1719,1,AMD,digitalocean,1.0,LinuxDreams,#AMD,RT @LinuxReviews: #Linux architect Linus Torva...,1.282227e+18,LinuxDreams,-0.3612,Technology,AMD,0,0
2,2020-07-12 08:11:41,AMD,69,135,AMD,Amsterdam,1.0,LinuxReviews,#AMD,"#Linux architect Linus Torvalds: AVX512 Is ""A ...",1.282226e+18,LinuxReviews,-0.3612,Technology,AMD,0,0
3,2020-07-12 02:22:50,AMD,34,155,AMD,San Francisco,0.0,NdrewGarcia,#AMD,#AMD stuck in a range box chart https://t.co/5...,1.282138e+18,Encino_Man,-0.2500,Technology,AMD,0,0
4,2020-07-11 23:58:44,AMD,802,730,AMD,"New Jersey, USA",0.0,Roger_Clinton1,#AMD,$AMD Epyc Milan Leak – Three early Genesis sam...,1.282102e+18,Roger Ocasio-Clinton,-0.3400,Technology,AMD,0,2


In [123]:
performance_metrics(df_tweets_bigtech_10ksample, "sentiment", "sentiment_prediction_finetuned")

Accuracy: 0.5633684210526316
Precision: 0.5890056459480304
Recall: 0.5633684210526316
F1-Score: 0.48415694314369734


In [ ]:
# comparison with not fine-tuned model on tweets-bigtech-data
df_tweets_bigtech_10ksample["sentiment_prediction"] = df_tweets_bigtech_10ksample["text"].apply(lambda x: sentiment_score(x[:500], tokenizer, model))

In [22]:
performance_metrics(df_tweets_bigtech_10ksample, "sentiment", "sentiment_prediction")

Accuracy: 0.5675789473684211
Precision: 0.6583776609118885
Recall: 0.5675789473684211
F1-Score: 0.5742206759300184
